In [32]:
import tensorflow as tf
import numpy as np

In [6]:
hello_constant = tf.constant("Hello World!",dtype=tf.string)
with tf.Session() as sess:
    output=sess.run(hello_constant)
    print(output)

b'Hello World!'


In [ ]:
# A is 0 dimensional int32 tensor
A = tf.constant(1234)
# B is 1 deminsional int32 tensor
B = tf.constant([123,456,789])
# C is 2 dimensional int32 tensor 
C = tf.constant([[123,456,789],[222,333,444]])


In [3]:
x=tf.placeholder(tf.string)
with tf.Session() as sess:
    output = sess.run(x,feed_dict={x:"Hello World!"})
    print(output)

Hello World!


In [10]:
x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)
s = tf.placeholder(tf.int32)
result = s*(x-y)
with tf.Session() as sess:
    output=sess.run(result,feed_dict={x:3,y:1,s:5})
    print(output)

10


In [17]:
x=10
y=2
z=x/y -1
m=tf.divide(x,y)-tf.constant(1,dtype=tf.float64)
with tf.Session() as sess:
    output=sess.run(m)
    print(output)

4.0


In [25]:
x = tf.Variable(5)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(x)
    print(output)

5


In [31]:
n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features,n_labels)))
bias = tf.Variable(tf.zeros(n_labels))
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    output=sess.run(weights)
    ouput_b = sess.run(bias)
    print(output.shape)
    print(bias.shape)

(120, 5)
(5,)


In [33]:
np.exp(1.2)/(sum(np.exp([1.2,0.9,0.4])))

0.45659031819443779

In [34]:
tf.reduce_sum([1,2,3,4])

<tf.Tensor 'Sum:0' shape=() dtype=int32>

In [35]:
softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

# TODO: Print cross entropy from session
cross_entropy = -tf.reduce_sum(tf.multiply(tf.log(softmax),one_hot))
with tf.Session() as sess:
    output = sess.run(cross_entropy,feed_dict={softmax:softmax_data,one_hot:one_hot_data})
    print(output)

0.356675


In [39]:
50000%128

80

In [38]:
50000-(390*128)

80

In [40]:
example_features = [
    ['F11','F12','F13','F14'],
    ['F21','F22','F23','F24'],
    ['F31','F32','F33','F34'],
    ['F41','F42','F43','F44']]

In [50]:
example_features[0:]

[['F11', 'F12', 'F13', 'F14'],
 ['F21', 'F22', 'F23', 'F24'],
 ['F31', 'F32', 'F33', 'F34'],
 ['F41', 'F42', 'F43', 'F44']]

# Mini-Batch Algorithm

In [52]:
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    # TODO: Implement batching
    batch = []
    n_b = len(features)/int(batch_size)
    int_nb = int(n_b)
    for k in range(int_nb):
        start = k*batch_size
        end = (k+1)*batch_size
        out=features[start:end]
        lout = labels[start:end]
        batch.append([out,lout])
    if len(features)%batch_size != 0:
        output = features[(len(features)-(len(features)%batch_size)): ]
        loutput = labels[(len(features)-(len(features)%batch_size)): ]
        batch.append([output,loutput])
    return batch

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
# Helper function created in Mini-batching section


def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Import MNIST data
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Features and Labels
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))

# Logits - xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

batch_size = 128
epochs = 10
learn_rate = 0.001

train_batches = batches(batch_size, train_features, train_labels)

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

In [59]:
import os
os.getcwd()

'/Users/unnatsingh/Udacity_dp_nanodegree/neural_network/deep-learning'